In [1]:
from os import scandir, getcwd
import pandas as pd
import json
import collections
import re
import itertools
from nltk.corpus import stopwords

In [2]:
files_path=getcwd()
# file = 'tweets_labeled_full.xlsx'
file = 'tweets_labeled.xlsx'

In [3]:
ds = pd.read_excel(file)
ds

,text,sentiment
0,NaN,3.0
1,mirando como se le cae el ojo a a dietrich cua...,1.0
2,lo gracioso es que cada vez que aumentan diet...,0.0
3,último momento dietrich anuncio aumentos ...,2.0
4,eeuu encabeza por lejos la lista de muertos po...,3.0
5,en macri ya planteaba que las fuerzas armadas...,0.0
6,me gusto un video de bonasso se harto de los ...,3.0
7,el gato que corrio a las ratas de casa rosada...,1.0
8,y este delincuente que es dueño de todo m...,0.0
9,se pinto la cara macri anuncio que las fuerza...,2.0


In [4]:
ds.count()

text         10523
sentiment     8002
dtype: int64

In [5]:
# df_columns = ds.-drop(['id', 'coordinates', 'place','lang','source','favorite_count','retweet_count'], axis=1)
df_clean1 = ds
df_clean1 = df_clean1.drop_duplicates(['text'], keep='first')
df_clean1 = df_clean1.dropna()
df_clean1

,text,sentiment
1,mirando como se le cae el ojo a a dietrich cua...,1.0
2,lo gracioso es que cada vez que aumentan diet...,0.0
3,último momento dietrich anuncio aumentos ...,2.0
4,eeuu encabeza por lejos la lista de muertos po...,3.0
5,en macri ya planteaba que las fuerzas armadas...,0.0
6,me gusto un video de bonasso se harto de los ...,3.0
7,el gato que corrio a las ratas de casa rosada...,1.0
8,y este delincuente que es dueño de todo m...,0.0
9,se pinto la cara macri anuncio que las fuerza...,2.0
10,el ex vicepresidente amado boudou fue condenad...,2.0


In [6]:
def preprocesamiento(row):
    print(row['text'])
    replaced = re.sub("(?:^|\s)[＃#]{1}(\w+)",'',str(row['text']) )
    replaced_wo_user= re.sub("(?:^|\s)[＠ @]{1}([^\s#<>[\]|{}]+)", '', replaced)
    replaced_links=re.sub("/[\w.-]*",'',replaced_wo_user) 
    replaced_links = replaced_links.replace("\r"," ")
    replaced_links = replaced_links.replace("\n"," ")
    replaced_links = ''.join([i for i in replaced_links if not i.isdigit()])
    a,b = 'áéíóúüàèìòù','aeiouuaeiou'
    trans = str.maketrans(a,b)
    replaced_links = replaced_links.translate(trans)    
    replaced_links = replaced_links.translate(str.maketrans('','','1234567890'))
    replaced_links = replaced_links.replace('[^\w\s]','')   
    
    return replaced_links   

In [7]:
# df_clean1 = df_columns
df_clean1['text'] = df_clean1.apply(preprocesamiento, axis=1)

mirando como se le cae el ojo a a dietrich cuando miente
 lo gracioso es que cada vez que aumentan dietrich te dice que es el ultimo aumento del año y a los dos meses vuelven a aumentar
último momento      dietrich anuncio aumentos en colectivos y trenes      
eeuu encabeza por lejos la lista de muertos por consumo de drogas y tiene el  de los asesinatos seriales registr 
en  macri ya planteaba que las fuerzas armadas tenian que salir a la calle       años despues siendo presidente lo impone por decreto
me gusto un video de  bonasso se harto de los intratables y estallo el tema es serio no es
 el gato que corrio a las ratas de casa rosada que raro que cfk no se haya robado el baston presidencial
     y este delincuente que es dueño de todo moron para cuando en cana   
se pinto la cara  macri anuncio que las fuerzas armadas comenzaran a intervenir en seguridad interior  via
el ex vicepresidente amado boudou fue condenado a  años y diez meses de prision por el delito de cohecho durante l

 voto la expropiacion de ciccone para salvar a boudou   estos son los miembros del    
 ham
la justicia en tiempos de cambiemos se llama injusticia   boudou paga por quitarles las afjp
no a la reforma restauracion reorganizacion ni una mierda de las 
macritipps cambiamos  y  camas diarias en el hospital  perchas y quedas bien con san cayetano y el diablo vamos todavianoesley    
si no son delincuentes terroristas narcotraficantes o agitadores de la paz interior xq le tienen tanto panico a las fuerzas armadas si la usan para controlar la inseguridad cual seria el problema oponerse por oponerse no les molesta o preocupa vivir sitiados x el crimen
un periodista acreditado en casa rosada tenia pedido de captura fue detenido  por silvia mercado  
macri rompera por decreto el consenso democratico sobre el rol de las fuerzas armadas 
el trending topic hoy en argentina por la ley de extincion de dominio claramente los argentinos estan hartos de la corrupcion
me preocupa quien cuidara del medan

      primer ataque terrorista al gobierno de      no nos van a intimidar el domingo masivamente a las
ahora  dietrich el sistema de transporte tiene costos por eso a partir del  de agosto aumentara en  por mes el boleto de colectivo 
 es necesario buscar y compartir los videos de agustin rossi y de todos los integrantes del bloque del frente para la victoria defendiendo a boudou y tratando de loca y de operadora politica cuando se debatia la expropiacion de ciccone y el juicio politico a boudou
 sistema necesita de las fuerzas armadas porque el modelo no cierra sin represion   el helicopte 
 esa verga no sirve para nada  dinero perdido en material y personal      para nada mant 
macri anuncia una reforma de las fuerzas armadas argentinas intervendran en la seguridad interior    
parece que hay acuerdo para proponer otra cosa y macri anuncia la depenalizacion en el codigo penal pero notras vams a seguir por y porque sino seguiremos muriendo
 ich nehme wetten an
にらんだ つもりは ないのだがな
 la ing

 jajajaja flase mal
estipula que el juez podra disponer que la pena se deje en suspenso o eximirla de ella teniendo en cuenta los motivos que la impulsaron a cometer el hecho 
las no estan para la seguridad interna bastantes desafios tienen por delante que no alcanzan a abordar en la defensa de la soberania
es barbaro que vuelvan las fuerzas armadas kretina krapula y que algún día te vayas del país  con tus vientos estafadores y ladrona del estado igual que cfk nunca fueron  mil  no llegan a seis mil y la mirad viven regio en españa o venezuela a que en eeuu 
no a las dos cosas    no es perseguido politico es un delincuente    y es cierto no le perdonamos el choreo de las afjp era dinero q teniamos ahorrados los contribuyentes x una ley que ellos mismos habian votado años atras   fue otra caja del kirchnerismo 
que pasquin sos clarin no era doctrina k era doctrina de la democracia que las fuerzas armadas no intervengan en seguridad interior infames y mentirosos 
argentina macri anuncio

veo armar las tribunas en la av brasil con toda normalidad seria muy triste ver a nuestras fuerzas armadas denigrarse al rendir honores a tanto delincuente seria muy triste tambien que la gente vaya a ese desfile   la ciudadania deberia impedir tremenda humillacion 
amado boudou fue condenado a  años y  meses de prision y quedo detenido 
los que militan el sepan que antes del kirchnerismo nadie hablaba del militantenadie militaba absolutamente nada      gracias a nestor y cristina recuperamos la política incluso para uds se sumaran y hoy participen como militantes      milita el timbre
el boleto de tren y colectivo en el area metropolitana de buenos aires tendra nuevos aumentos el ministro de transporte guillermo dietrich  anunciara esta tarde el nuevo esquema tarifario para el sector
yo entiendo que hay que odiar a boudou por ladron pero se llama amado se llama amado
 gente sigue sin creer la pavada que dijo para atacar a la izquierda de del caño y myriam bregman
mas ajuste dietrich s

 les cago el choreo de las afjp clarin y secuaces los verdadeos chorros de este pais
asignan a las fuerzas armadas tareas de seguridad interior principalmente a una guerra contra el narcotrafico es un decreto claramente inconstitucional espero que toda esta locura quede sin efecto lo antes posible
 ponele boludo 
 se hace con esto  algun fiscal de turno  jueza servini 
me encanta la excusa que da boudou por la causa el no fue el responsable de robar la maquina de imprimir billetes para la corona nestor y cristina sino que la culpa la tienen las afjp ja
 decidiste a quien vas a votar en provincia rt
 y son muchos mas inmuebles todos manejados x ramon lanus g amigo de peña compañero de hazañas de dietrich
es importante que colaboren con la seguridad interior indico el presidente las frases mas salientes del madatario 
 no pa yo no te tengo que argumentar nadaboudou preso y me parece muuuuy bien a vos no no decime porque no que me quiero cagar de risa a ver
el senador se refirio a la refo

In [8]:
df_clean = df_clean1
df_clean["text"] = df_clean['text'].str.replace('[^\w\s]','')

In [9]:
df_clean['text'] = df_clean['text'].str.lower()
df_clean

,text,sentiment
1,mirando como se le cae el ojo a a dietrich cua...,1.0
2,gracioso es que cada vez que aumentan dietric...,0.0
3,ultimo momento anuncio aumentos en colecti...,2.0
4,eeuu encabeza por lejos la lista de muertos po...,3.0
5,en ya planteaba que las fuerzas armadas tenian...,0.0
6,me gusto un video de se harto de los intratabl...,3.0
7,gato que corrio a las ratas de casa rosada qu...,1.0
8,este delincuente que es dueño de todo moro...,0.0
9,se pinto la cara anuncio que las fuerzas armad...,2.0
10,el ex vicepresidente amado boudou fue condenad...,2.0


In [10]:
df_clean = df_clean.reset_index()
df_clean=df_clean[['text','sentiment']]
df_clean

,text,sentiment
0,mirando como se le cae el ojo a a dietrich cua...,1.0
1,gracioso es que cada vez que aumentan dietric...,0.0
2,ultimo momento anuncio aumentos en colecti...,2.0
3,eeuu encabeza por lejos la lista de muertos po...,3.0
4,en ya planteaba que las fuerzas armadas tenian...,0.0
5,me gusto un video de se harto de los intratabl...,3.0
6,gato que corrio a las ratas de casa rosada qu...,1.0
7,este delincuente que es dueño de todo moro...,0.0
8,se pinto la cara anuncio que las fuerzas armad...,2.0
9,el ex vicepresidente amado boudou fue condenad...,2.0


In [11]:
df_clean = df_clean[df_clean.sentiment < 3]
df_clean

,text,sentiment
0,mirando como se le cae el ojo a a dietrich cua...,1.0
1,gracioso es que cada vez que aumentan dietric...,0.0
2,ultimo momento anuncio aumentos en colecti...,2.0
4,en ya planteaba que las fuerzas armadas tenian...,0.0
6,gato que corrio a las ratas de casa rosada qu...,1.0
7,este delincuente que es dueño de todo moro...,0.0
8,se pinto la cara anuncio que las fuerzas armad...,2.0
9,el ex vicepresidente amado boudou fue condenad...,2.0
10,el festejo del globoludo por la noticia de es ...,0.0
11,caso ciccone boudou condenado a prision,2.0


In [12]:
df_clean.to_excel('tweets_preprocessed.xlsx')